In [1]:
import pickle
import numpy as np
import tensorflow
from numpy.linalg import norm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
import cv2

In [26]:
pip install opencv-python

     -------------------------------------- 39.0/39.0 MB 812.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
feature_list = np.array(pickle.load(open('embeddings.pkl','rb')))

In [3]:
filenames = pickle.load(open('filenames.pkl','rb'))

In [4]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_max_pooling2d (Global (None, 2048)              0         
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [44]:
img= image.load_img('sample/1623.jpg',target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array,axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)

In [45]:
neighbors = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean')
neighbors.fit(feature_list)

NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=6)

In [46]:
distances,indices = neighbors.kneighbors([normalized_result])

In [47]:
print(indices)

[[ 5357  9727 12623  2037  8779 10949]]


In [48]:
for file in indices[0][1:6]:
    temp_img= cv2.imread((filenames[file]))
    cv2.imshow('output',cv2.resize(temp_img,(512,512)))
    cv2.waitKey(0)